# Exercises: Points

Build your understanding of the concepts from `core_03_points.ipynb`.

These exercises are designed so you **write the code yourself**. Each exercise has
an empty code cell — fill it in, run it, and check whether the assertions pass.
If you get stuck, refer back to the tutorial notebook.

In [1]:
import numpy as np
import warp as wp

wp.config.quiet = True
wp.init()

---
## Exercise 1: The Neighbor Problem

### Think first

Suppose you have 10,000 particles and need to find which ones are close to each other.
The naive approach checks every pair — that's ~50 million distance calculations per step.

**Question:** Why is a hash grid faster? What does it trade off to get that speed?

*(Think about it before reading on. There's no code to write here.)*

<details>
<summary>Discussion</summary>

A hash grid divides space into cells. Each particle is assigned to a cell based on its position.
When querying neighbors, you only check particles in nearby cells — not all 10,000.
This turns an O(N²) problem into roughly O(N) in practice.

The trade-off: you must **choose a cell size** and **rebuild the grid** every time particles
move. If the cell size is wrong, you either check too many cells (too small) or put too many
particles in one cell (too large). The cell size should roughly match your query radius.
</details>

---
## Exercise 2: Build a Hash Grid and Count Neighbors

Write a kernel that counts how many neighbors each point has within a given radius.

You need to:
1. Create a `wp.HashGrid` and call `grid.build(points, cell_size)`
2. In the kernel, use `wp.hash_grid_point_id(grid, tid)` to get the real point index
3. Use `wp.hash_grid_query(grid, position, radius)` to get an iterator over nearby points
4. Loop with `for index in neighbors:`, skip `index == i`, and count

**Test data:** 5 points — three clustered near the origin, one nearby, one very far away.
Use a query radius of `2.0`.

In [21]:
# Your code here.
#
# Write a @wp.kernel called `count_neighbors` that takes:
#   grid: wp.uint64
#   positions: wp.array(dtype=wp.vec3)
#   radius: float
#   counts: wp.array(dtype=wp.int32)
#
# Then create the points, build the grid, launch the kernel, and check the assertions below.

@wp.kernel
def count_neightbors(grid: wp.uint64, positions: wp.array(dtype=wp.vec3), radius: float, counts: wp.array(dtype=wp.int32)):
    tid = wp.tid()
    point_id = wp.hash_grid_point_id(grid, tid)
    point = positions[point_id]
    #print("=======")
    #print(point)
    #print("----------")
    neighbors = wp.hash_grid_query(grid, point, radius)
    for index in neighbors:
        if index == point_id:
            continue
        counts[point_id] += 1

grid = wp.HashGrid(128, 128, 128)
p0 = wp.vec3f(0.0, 0.0, 0.5)
p1 = wp.vec3f(0.0, 0.5, 0.0)
p2 = wp.vec3f(0.5, 0.0, 0.0)
p3 = wp.vec3f(1.5, 0.0, 0.0)
p4 = wp.vec3f(10.0, 20.0, 15.0)
points = wp.array([p0, p1, p2, p3, p4], dtype=wp.vec3f)
grid.build(points=points, radius=1.0)
counts = wp.zeros(shape=points.shape, dtype=wp.int32)
radius = 2.0
wp.launch(count_neightbors, dim=points.shape, inputs=(grid.id, points, radius), outputs=(counts,))
print(counts)

[3 3 3 3 0]


In [22]:
# --- Assertions (do not modify) ---
result = counts.numpy()
print("Neighbor counts:", result)
# The isolated point should have 0 neighbors.
assert result[4] == 0, f"Expected isolated point to have 0 neighbors, got {result[4]}"
# The three clustered points should each see at least 2 neighbors.
assert result[0] >= 2
assert result[1] >= 2
assert result[2] >= 2
print("Passed!")

Neighbor counts: [3 3 3 3 0]
Passed!


---
## Exercise 3: Break It — Cell Size

Using your working code from Exercise 2, experiment with the `cell_size` parameter
passed to `grid.build()`.

Try these values and observe the neighbor counts:
1. `cell_size = 0.001` (much smaller than the query radius)
2. `cell_size = 100.0` (much larger than the query radius)
3. `cell_size = 2.0` (matching the query radius)

**Question:** Do the results change? Which cell size is most *efficient* and why?

<details>
<summary>Discussion</summary>

The results (neighbor counts) should be the **same** in all cases — correctness doesn't depend
on cell size. But performance does:

- **Too small** (0.001): Each query must check thousands of tiny cells to cover the radius.
  Correct but slow.
- **Too large** (100.0): All points end up in one cell, so every query checks every point.
  Correct but degenerates to O(N²).
- **Matching the radius** (~2.0): Each query checks ~27 cells (3x3x3 neighborhood).
  This is the sweet spot.
</details>

In [ ]:
# Experiment here: copy your code from Exercise 2 and try different cell_size values.
# Compare the neighbor counts.



---
## Exercise 4: Symplectic Euler Integration

### Think first

The tutorial updates velocity *first*, then uses the **new** velocity to update position:
```
v_new = v + (force * inv_mass + gravity) * dt
x_new = x + v_new * dt       # <-- uses v_new, not v
```

**Question:** What would happen if you used the *old* velocity instead (`x_new = x + v * dt`)?
Why does the order matter?

<details>
<summary>Discussion</summary>

Using the old velocity is standard (explicit) Euler. It works, but it tends to **gain energy**
over time — a bouncing ball bounces higher and higher. Symplectic Euler (updating v first,
then using it for x) conserves energy much better over long simulations. It's a one-line
difference with a big practical impact.
</details>

### Now write it

Write an `integrate` kernel that takes position, velocity, and force arrays (all `wp.vec3`),
plus `gravity` (`wp.vec3`), `dt` (`float`), and `inv_mass` (`float`). Update both arrays in-place
using symplectic Euler.

Test: drop a particle from `y=10` with zero initial velocity under gravity `(0, -9.8, 0)`.
After 10 steps of `dt=0.1`, it should have fallen.

In [ ]:
# Your code here.
#
# Write a @wp.kernel called `integrate` and test it.



In [ ]:
# --- Assertions (do not modify) ---
pos = positions.numpy()[0]
vel = velocities.numpy()[0]
print(f"Position: {pos}")
print(f"Velocity: {vel}")
assert pos[1] < 10.0, "Particle should have fallen"
assert vel[1] < 0.0, "Velocity should be downward"
print("Passed!")

---
## Exercise 5: Particle Repulsion Simulation

Now combine everything into a mini simulation. This is the core loop from the tutorial,
stripped down to the essentials.

Write a `compute_repulsion` kernel that:
1. Uses `wp.hash_grid_point_id()` to get the real point index
2. Uses `wp.hash_grid_query()` to find neighbors
3. For each neighbor: computes `overlap = 2 * radius - distance`
4. If `overlap > 0` and `distance > 0`: adds a force along the direction *away* from the neighbor,
   proportional to `overlap * k_repel`

Then write the simulation loop:
```
for each step:
    grid.build(positions, cell_size)
    launch compute_repulsion
    launch integrate
```

**Test data:** 3 particles at `x = 0.0, 0.1, 0.2` with `radius = 0.1`. They overlap,
so repulsion should push them apart. Use `k_repel = 1000.0`, `dt = 0.001`, no gravity,
100 steps.

In [ ]:
# Your code here.
#
# Write @wp.kernel compute_repulsion, set up the particles and grid,
# run the simulation loop, then check the assertion below.
#
# Reuse your `integrate` kernel from Exercise 4.



In [ ]:
# --- Assertions (do not modify) ---
final_pos = positions.numpy()
initial_span = 0.2  # initial x range: 0.2 - 0.0
final_span = final_pos[-1, 0] - final_pos[0, 0]
print(f"Final positions: {final_pos}")
print(f"Initial span: {initial_span:.4f}, Final span: {final_span:.4f}")
assert final_span > initial_span, "Particles should have spread apart"
print("Passed!")

---
## Exercise 6: Break It — Simulation Parameters

Take your working simulation from Exercise 5 and try each of these changes **one at a time**.
Predict what will happen, then run it.

1. **Remove `index != i` check** — what goes wrong?
2. **Set `dt = 1.0`** (instead of 0.001) — does the simulation still work?
3. **Don't rebuild the grid** (move `grid.build()` before the loop) — does it still give correct results?
4. **Set `k_repel = 0.01`** — what changes?

<details>
<summary>Discussion</summary>

1. **No self-skip:** Each particle computes overlap with itself (distance=0), producing
   division by zero or huge forces. The simulation explodes or produces NaN.

2. **Large dt:** The simulation becomes unstable — particles overshoot, pass through each other,
   and fly off to infinity. This is why the tutorial uses many substeps with small dt.

3. **Stale grid:** The grid still reflects the *initial* positions, so neighbor queries
   become incorrect as particles move. For small movements it roughly works; for large
   movements it misses neighbors entirely.

4. **Weak repulsion:** Particles barely move — the force is too small relative to their overlap.
   They stay nearly in place.
</details>

In [ ]:
# Experiment here: copy your simulation from Exercise 5 and try the modifications above.

